In [134]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [135]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [136]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [137]:
# # replace NAN's
# def rmNAN(df, col):
#     df.col = df.col.fillna(0)

In [138]:
sallyMetrics = loadMetricData('sallyKung', "metrics")
sallyMetrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-31 23:55:00,74,0,0.9,0.000058,90.5,88.7
1436,2016-05-31 23:56:00,74,0,0.9,0.000059,90.5,88.7
1437,2016-05-31 23:57:00,74,0,1.0,0.000059,91.4,89.6
1438,2016-05-31 23:58:00,74,0,0.9,0.000060,91.4,89.6
1439,2016-05-31 23:59:00,75,0,0.9,0.000060,91.4,90.5


In [139]:
sallyMetrics.dtypes

timestamp     object
heartrate    float64
steps          int64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [140]:
# clean metric data

sallyMetrics.timestamp = pd.to_datetime(sallyMetrics.timestamp)

sallyMetrics.heartrate = sallyMetrics.heartrate.fillna(0)
sallyMetrics.steps = sallyMetrics.steps.fillna(0)
sallyMetrics.calories = sallyMetrics.calories.fillna(0)
sallyMetrics.gsr = sallyMetrics.gsr.fillna(0)
sallyMetrics.skintemp = sallyMetrics.skintemp.fillna(0)
sallyMetrics.airtemp = sallyMetrics.airtemp.fillna(0)

sallyMetrics.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
0,2016-05-02 00:00:00,0,4,1.709,0,0,0
1,2016-05-02 00:01:00,0,4,1.709,0,0,0
2,2016-05-02 00:02:00,0,4,1.709,0,0,0
3,2016-05-02 00:03:00,0,4,1.709,0,0,0
4,2016-05-02 00:04:00,0,4,1.709,0,0,0


In [141]:
sallyActivities = loadMetricData('sallyKung', "activities")
sallyActivities.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,type,actual seconds,steps,calories,minutes,heart rate avg,heart rate min,heart rate max,state,version,id
1,2016-06-06 06:54:29,2016-06-06T06:54:29Z,Asia/Shanghai,480,2016-06-06 07:04:43,2016-06-06T07:04:43Z,Asia/Shanghai,480,walk,528,912,30.600000,10.233333,110.592804,NaN,NaN,complete,1,walk__2016-06-06T065429
2,2016-06-06 09:32:27,2016-06-06T09:32:27Z,Asia/Shanghai,480,2016-06-06 10:02:15,2016-06-06T10:02:15Z,Asia/Shanghai,480,walk,1251,2152,68.099998,29.800000,112.898483,NaN,NaN,complete,1,walk__2016-06-06T093227
3,2016-06-06 12:30:14,2016-06-06T12:30:14Z,Asia/Shanghai,480,2016-06-06 12:38:08,2016-06-06T12:38:08Z,Asia/Shanghai,480,walk,278,453,8.500000,7.900000,96.730217,NaN,NaN,complete,1,walk__2016-06-06T123014
4,2016-06-06 12:58:30,2016-06-06T12:58:30Z,Asia/Shanghai,480,2016-06-06 13:05:33,2016-06-06T13:05:33Z,Asia/Shanghai,480,walk,348,621,11.800000,7.050000,100.965515,NaN,NaN,complete,1,walk__2016-06-06T125830
5,2016-06-06 13:23:07,2016-06-06T13:23:07Z,Asia/Shanghai,480,2016-06-06 13:27:43,2016-06-06T13:27:43Z,Asia/Shanghai,480,walk,233,419,6.100000,4.600000,93.738197,NaN,NaN,complete,1,walk__2016-06-06T132307


In [142]:
sallySleep = loadMetricData('sallyKung', "sleep")
sallySleep.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,light mins,deep mins,...,toss turns,type,actual seconds,calories,heart rate avg,heart rate min,heart rate max,state,version,id
1,2016-06-02 16:56:58,2016-06-02T16:56:58Z,Asia/Shanghai,480,2016-06-03 00:49:45,2016-06-03T00:49:45Z,Asia/Shanghai,480,252,119,...,36,sleep,28320,437.200012,59.062733,NaN,NaN,complete,2,sleep__2016-06-02T165658
0,2016-06-03 17:07:29,2016-06-03T17:07:29Z,Asia/Shanghai,480,2016-06-04 02:03:05,2016-06-04T02:03:05Z,Asia/Shanghai,480,326,83,...,44,sleep,32100,495.100006,67.824936,NaN,NaN,complete,2,sleep__2016-06-03T170729
0,2016-06-04 18:36:17,2016-06-04T18:36:17Z,Asia/Shanghai,480,2016-06-05 03:51:17,2016-06-05T03:51:17Z,Asia/Shanghai,480,232,165,...,46,sleep,33300,513.400024,67.653275,NaN,NaN,complete,2,sleep__2016-06-04T183617
0,2016-06-05 18:20:50,2016-06-05T18:20:50Z,Asia/Shanghai,480,2016-06-06 02:19:31,2016-06-06T02:19:31Z,Asia/Shanghai,480,277,94,...,52,sleep,28680,443.100006,61.587814,NaN,NaN,complete,2,sleep__2016-06-05T182050
0,2016-06-06 02:51:35,2016-06-06T02:51:35Z,Asia/Shanghai,480,2016-06-06 04:47:35,2016-06-06T04:47:35Z,Asia/Shanghai,480,62,32,...,11,sleep,6960,107.400002,60.152729,NaN,NaN,complete,2,sleep__2016-06-06T025135


In [143]:
sallySleep.dtypes

start time              object
start time ISO          object
start time timezone     object
start time offset      float64
end time                object
end time ISO            object
end time timezone       object
end time offset        float64
light mins             float64
deep mins              float64
rem mins               float64
interruption mins      float64
unknown mins           float64
interruptions          float64
toss turns             float64
type                    object
actual seconds         float64
calories               float64
heart rate avg         float64
heart rate min         float64
heart rate max         float64
state                   object
version                float64
id                      object
dtype: object

In [144]:
sallyJournal = loadMetricData('sallyKung', "journal")
# sallyJournal.tail(5)

In [145]:
# sallyJournal.dtypes

In [146]:
# data= barbJournal.iloc[:0]
# clean metric data
sallyJournal["starttime"]= pd.to_datetime(sallyJournal.starttime)

# data= barbJournal.iloc[:1]
sallyJournal["endtime"]= pd.to_datetime(sallyJournal.endtime)


# sarahJournal.qualitative = sarahJournal.qualitative.convert_objects(convert_numeric=True)
# sarahJournal.excited = sarahJournal.excited.convert_objects(convert_numeric=True)
# sarahJournal.happy = sarahJournal.happy.convert_objects(convert_numeric=True)

sallyJournal.qualitative = sallyJournal.qualitative.fillna(0)
sallyJournal.excited = sallyJournal.excited.fillna(0)
sallyJournal.happy = sallyJournal.happy.fillna(0)
sallyJournal.calm = sallyJournal.calm.fillna(0)
sallyJournal.anxious = sallyJournal.anxious.fillna(0)
sallyJournal.sad = sallyJournal.sad.fillna(0)
sallyJournal.angry = sallyJournal.angry.fillna(0)
sallyJournal.hungry = sallyJournal.hungry.fillna(0)
sallyJournal.tired = sallyJournal.tired.fillna(0)
sallyJournal.bored = sallyJournal.bored.fillna(0)

print(sallyJournal.shape)
# sallyJournal.head(5)
# sallyJournal.tail(5)

(292, 24)


In [147]:

sallyJournal['boolexcited'] =  sallyJournal['excited'] > 0
sallyJournal['boolhappy'] =  sallyJournal['happy'] > 0
sallyJournal['boolcalm'] =  sallyJournal['calm'] > 0
sallyJournal['boolanxious'] =  sallyJournal['anxious'] > 0
sallyJournal['boolsad'] =  sallyJournal['sad'] > 0
sallyJournal['boolangry'] =  sallyJournal['angry'] > 0
sallyJournal['booltired'] =  sallyJournal['tired'] > 0
sallyJournal['boolhungry'] =  sallyJournal['hungry'] > 0
sallyJournal['boolbored'] =  sallyJournal['bored'] > 0
sallyJournal['exhaca'] =  sallyJournal['exhaca'] > 0
sallyJournal['ansaan'] =  sallyJournal['ansaan'] > 0
sallyJournal['exha'] =  sallyJournal['exha'] > 0
sallyJournal['haca'] =  sallyJournal['haca'] > 0
sallyJournal['anan'] =  sallyJournal['anan'] > 0
sallyJournal['ansa'] =  sallyJournal['anan'] > 0

    
boolexcited = sallyJournal.iloc[:, 18].values.reshape(sallyJournal.shape[0], 1)
boolhappy = sallyJournal.iloc[:, 19].values.reshape(sallyJournal.shape[0], 1)
boolcalm = sallyJournal.iloc[:, 20].values.reshape(sallyJournal.shape[0], 1)
boolanxious = sallyJournal.iloc[:, 21].values.reshape(sallyJournal.shape[0], 1)
boolsad = sallyJournal.iloc[:, 22].values.reshape(sallyJournal.shape[0], 1)
boolangry = sallyJournal.iloc[:, 23].values.reshape(sallyJournal.shape[0], 1)
booltired = sallyJournal.iloc[:, 24].values.reshape(sallyJournal.shape[0], 1)
boolhungry = sallyJournal.iloc[:, 25].values.reshape(sallyJournal.shape[0], 1)
boolbored = sallyJournal.iloc[:, 26].values.reshape(sallyJournal.shape[0], 1)
exhaca = sallyJournal.iloc[:, 27].values.reshape(sallyJournal.shape[0], 1)
ansaan = sallyJournal.iloc[:, 28].values.reshape(sallyJournal.shape[0], 1)
exha = sallyJournal.iloc[:, 29].values.reshape(sallyJournal.shape[0], 1)
haca = sallyJournal.iloc[:, 30].values.reshape(sallyJournal.shape[0], 1)
anan = sallyJournal.iloc[:, 31].values.reshape(sallyJournal.shape[0], 1)



sallyJournal.head(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,ansa,boolexcited,boolhappy,boolcalm,boolanxious,boolsad,boolangry,booltired,boolhungry,boolbored
0,2016-05-02 13:25:00,2016-05-02 13:28:00,2016-05-02,1:25 PM,1:28 PM,Trying on watch,Barb's Office,Excited,8,8,...,False,True,True,False,False,False,False,False,False,False
1,2016-05-02 13:40:00,2016-05-02 13:48:00,2016-05-02,1:40 PM,1:48 PM,Editing photos for work,Barb's Office,Streeeeeessed,6,0,...,False,False,False,False,False,False,False,False,False,False
2,2016-05-02 13:48:00,2016-05-02 13:48:00,2016-05-02,1:48 PM,1:48 PM,Wafer fit perfectly on top of cup,Barb's Office,HAPPY,10,10,...,False,True,True,False,False,False,False,False,False,False
3,2016-05-02 14:31:00,2016-05-02 14:50:00,2016-05-02,2:31 PM,2:50 PM,Sorting through magnetic pins,Barb's Office,Anxious,5,0,...,True,False,False,False,True,False,True,False,False,False
4,2016-05-02 15:10:00,2016-05-02 15:14:00,2016-05-02,3:10 PM,3:14 PM,Trying to figure out where photos went,Barb's Office,"ANXIOUS, CONFUSED",9,0,...,True,False,False,False,True,False,False,False,False,False


In [148]:
# sallyJournal.dtypes

In [149]:
# sallyJournal.excited.convert_objects(convert_numeric=True)

In [150]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [151]:
# indicator for walking
totalData = makeIndicators('excitement', sallyMetrics, sallyJournal, 0, 1, 24)
totalData = makeIndicators('happy', sallyMetrics, sallyJournal, 0, 1, 25)
totalData = makeIndicators('calm', sallyMetrics, sallyJournal, 0, 1, 26)
totalData = makeIndicators('anxious', sallyMetrics, sallyJournal, 0, 1, 27)
totalData = makeIndicators('sad', sallyMetrics, sallyJournal, 0, 1, 28)
totalData = makeIndicators('angry', sallyMetrics, sallyJournal, 0, 1, 29)
totalData = makeIndicators('hungry', sallyMetrics, sallyJournal, 0, 1, 30)
totalData = makeIndicators('tired', sallyMetrics, sallyJournal, 0, 1, 31)
totalData = makeIndicators('bored', sallyMetrics, sallyJournal, 0, 1, 32)


totalData = makeIndicators('exhaca', sallyMetrics, sallyJournal, 0, 1, 27)
totalData = makeIndicators('ansaan', sallyMetrics, sallyJournal, 0, 1, 28)

totalData = makeIndicators('exha', sallyMetrics, sallyJournal, 0, 1, 29)
totalData = makeIndicators('haca', sallyMetrics, sallyJournal, 0, 1, 30)
totalData = makeIndicators('anan', sallyMetrics, sallyJournal, 0, 1, 31)
totalData = makeIndicators('ansa', sallyMetrics, sallyJournal, 0, 1, 32)



In [152]:
print(totalData.describe())

          heartrate         steps      calories           gsr      skintemp  \
count  43920.000000  43920.000000  43920.000000  43920.000000  43920.000000   
mean      77.215255      7.071061      1.633281      0.009605     82.985501   
std       24.908041     23.241126      1.194595      0.163470     21.224387   
min        0.000000      0.000000      0.600000      0.000000      0.000000   
25%       65.000000      0.000000      1.000000      0.000054     85.100000   
50%       79.000000      0.000000      1.200000      0.000059     88.700000   
75%       92.000000      0.000000      1.709000      0.000072     90.500000   
max      169.000000    143.000000     12.000000     17.100000    100.400000   

            airtemp excitement      happy      calm    anxious    ...      \
count  43920.000000      43920      43920     43920      43920    ...       
mean      79.940501  0.0176002  0.0346995  0.123042  0.0623179    ...       
std       20.560615   0.131495    0.18302  0.328489   0.2

In [153]:
print(totalData.head())

            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-05-02 00:00:00          0      4     1.709    0         0        0   
1 2016-05-02 00:01:00          0      4     1.709    0         0        0   
2 2016-05-02 00:02:00          0      4     1.709    0         0        0   
3 2016-05-02 00:03:00          0      4     1.709    0         0        0   
4 2016-05-02 00:04:00          0      4     1.709    0         0        0   

  excitement  happy   calm  ...    angry hungry  tired  bored exhaca ansaan  \
0      False  False  False  ...    False  False  False  False  False  False   
1      False  False  False  ...    False  False  False  False  False  False   
2      False  False  False  ...    False  False  False  False  False  False   
3      False  False  False  ...    False  False  False  False  False  False   
4      False  False  False  ...    False  False  False  False  False  False   

    exha   haca   anan   ansa  
0  False  False  False  False 

In [154]:
totalData.to_csv('sallyTotal.csv')

In [155]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,angry,hungry,tired,bored,exhaca,ansaan,exha,haca,anan,ansa
0,2016-05-02 00:00:00,0,4,1.709,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2016-05-02 00:01:00,0,4,1.709,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2016-05-02 00:02:00,0,4,1.709,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2016-05-02 00:03:00,0,4,1.709,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2016-05-02 00:04:00,0,4,1.709,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [156]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute


print(totalData.shape)
print(totalData.head(5))

(43920, 26)
            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-05-02 00:00:00          0      4     1.709    0         0        0   
1 2016-05-02 00:01:00          0      4     1.709    0         0        0   
2 2016-05-02 00:02:00          0      4     1.709    0         0        0   
3 2016-05-02 00:03:00          0      4     1.709    0         0        0   
4 2016-05-02 00:04:00          0      4     1.709    0         0        0   

  excitement  happy   calm ... exhaca ansaan   exha   haca   anan   ansa  \
0      False  False  False ...  False  False  False  False  False  False   
1      False  False  False ...  False  False  False  False  False  False   
2      False  False  False ...  False  False  False  False  False  False   
3      False  False  False ...  False  False  False  False  False  False   
4      False  False  False ...  False  False  False  False  False  False   

  weekday month hour min  
0       0     5    0   0  
1       0     

In [157]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)

# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData['weekday'].values.reshape(totalData.shape[0], 1)
month = totalData['month'].values.reshape(totalData.shape[0], 1)
hour = totalData['hour'].values.reshape(totalData.shape[0], 1)
# curmin = totalData['min'].values.reshape(totalData.shape[0], 1)

print(hour)

[[ 0]
 [ 0]
 [ 0]
 ..., 
 [23]
 [23]
 [23]]


In [158]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp,
        weekday,
#         month,
        hour
#         curmin
    ), axis=1)

X.shape

(43920, 8)

In [159]:
# define y
y = totalData.excitement.values
y.shape

(43920,)

In [160]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [161]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [162]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

# Excited Model Results

In [163]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0173041894353 0.0182908318154
 Accuracy Rate  0.982695810565  0.981709168185
True Positives             0.0             0.0
True Negatives              --             1.0


(0.01829083181542198, 0.98170916818457798, 0.0, 1.0)

In [164]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0173041894353 0.0182908318154
      Accuracy Rate  0.982695810565  0.981709168185
     True Positives             0.0             0.0
     True Negatives              --             1.0


(0.01829083181542198, 0.98170916818457798, 0.0, 1.0)

In [165]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0173041894353 0.0182908318154
 Accuracy Rate  0.982695810565  0.981709168185
True Positives             0.0             0.0
True Negatives              --             1.0


(0.01829083181542198, 0.98170916818457798, 0.0, 1.0)

In [166]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0238095238095 0.0245901639344
 Accuracy Rate   0.97619047619  0.975409836066
True Positives 0.0131578947368 0.0207468879668
True Negatives              --  0.993196752996


(0.024590163934426229,
 0.97540983606557374,
 0.020746887966804978,
 0.99319675299574794)

In [167]:
# define y
y = totalData.happy.values
y.shape

(43920,)

In [168]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [169]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Happy Model Results

In [170]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set      Test Set   
-------------- -------------- ---------------
    Error Rate 0.053181108509 0.0598816029144
 Accuracy Rate 0.946818891491  0.940118397086
True Positives 0.109126984127  0.124031007752
True Negatives             --  0.973380726698


(0.059881602914389799,
 0.94011839708561018,
 0.12403100775193798,
 0.97338072669826226)

In [171]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set      Test Set   
------------------- -------------- ---------------
         Error Rate 0.057702315899 0.0638281724347
      Accuracy Rate 0.942297684101  0.936171827565
     True Positives 0.144841269841  0.160852713178
     True Negatives             --  0.967772511848


(0.06382817243472981,
 0.93617182756527018,
 0.16085271317829458,
 0.96777251184834123)

In [172]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0328194119178 0.0391621129326
 Accuracy Rate  0.967180588082  0.960837887067
True Positives             0.0             0.0
True Negatives              --             1.0


(0.039162112932604735, 0.96083788706739526, 0.0, 1.0)

In [173]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0547098620869 0.0600333940498
 Accuracy Rate  0.945290137913   0.93996660595
True Positives 0.0922619047619  0.110465116279
True Negatives              --  0.973775671406


(0.06003339404978749,
 0.93996660595021253,
 0.11046511627906977,
 0.97377567140600318)

In [174]:
# define y
y = totalData.calm.values
y.shape

(43920,)

In [175]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [176]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Calm Model Results

In [177]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set     Test Set   
-------------- --------------- --------------
    Error Rate  0.132285974499 0.128794778385
 Accuracy Rate  0.867714025501 0.871205221615
True Positives 0.0147058823529 0.016290726817
True Negatives              -- 0.989032815199


(0.12879477838494233,
 0.87120522161505765,
 0.016290726817042606,
 0.9890328151986183)

In [178]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate   0.129228467343   0.127276867031
      Accuracy Rate   0.870771532657   0.872723132969
     True Positives 0.00735294117647 0.00877192982456
     True Negatives               --   0.991796200345


(0.12727686703096538,
 0.87272313296903459,
 0.008771929824561403,
 0.99179620034542315)

In [179]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate   0.124414519906   0.121736490589
 Accuracy Rate   0.875585480094   0.878263509411
True Positives 0.00078781512605 0.00062656641604
True Negatives               --   0.999222797927


(0.1217364905889496,
 0.87826350941105036,
 0.00062656641604010022,
 0.99922279792746116)

In [180]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate   0.127211813687   0.124620522162
 Accuracy Rate   0.872788186313   0.875379477838
True Positives 0.00341386554622 0.00375939849624
True Negatives               --   0.995509499136


(0.12462052216150576,
 0.87537947783849424,
 0.0037593984962406013,
 0.99550949913644216)

In [181]:
# define y
y = totalData.anxious.values
y.shape

(43920,)

In [182]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [183]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Anxious Model Results

In [184]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.252179287015 0.258120825744
 Accuracy Rate 0.747820712985 0.741879174256
True Positives 0.517847629196 0.497674418605
True Negatives             -- 0.758931471257


(0.25812082574377654,
 0.74187917425622341,
 0.49767441860465117,
 0.75893147125690164)

In [185]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set     Test Set   
------------------- --------------- --------------
         Error Rate 0.0610525631017 0.065270188221
      Accuracy Rate  0.938947436898 0.934729811779
     True Positives             0.0            0.0
     True Negatives              --            1.0


(0.065270188221007899, 0.93472981177899206, 0.0, 1.0)

In [186]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0622235232891  0.0664086217365
 Accuracy Rate  0.937776476711   0.933591378264
True Positives 0.0042621204049 0.00348837209302
True Negatives              --   0.998538486522


(0.066408621736490592,
 0.93359137826350946,
 0.0034883720930232558,
 0.99853848652159793)

In [187]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0836260733802 0.0884942319369
 Accuracy Rate   0.91637392662  0.911505768063
True Positives  0.125199786894  0.123255813953
True Negatives              --  0.966547580383


(0.088494231936854892,
 0.91150576806314509,
 0.12325581395348838,
 0.96654758038324129)

In [188]:
# define y
y = totalData.sad.values
y.shape

(43920,)

In [189]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [190]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Sad Model Values

In [191]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00162633359355 0.00129022465088
 Accuracy Rate   0.998373666406   0.998709775349
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0012902246508803886, 0.99870977534911964, 0.0, 1.0)

In [192]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate 0.00169138693729 0.00144201578628
      Accuracy Rate   0.998308613063   0.998557984214
     True Positives              0.0              0.0
     True Negatives               --   0.999848012767


(0.0014420157862780813, 0.99855798421372188, 0.0, 0.99984801276692759)

In [193]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00162633359355 0.00129022465088
 Accuracy Rate   0.998373666406   0.998709775349
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0012902246508803886, 0.99870977534911964, 0.0, 1.0)

In [194]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00162633359355 0.00129022465088
 Accuracy Rate   0.998373666406   0.998709775349
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0012902246508803886, 0.99870977534911964, 0.0, 1.0)

In [195]:
# define y
y = totalData.angry.values
y.shape

(43920,)

In [196]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [197]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Angry Model Results

In [198]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0290463179807 0.0311930783242
 Accuracy Rate  0.970953682019  0.968806921676
True Positives             0.0             0.0
True Negatives              --             1.0


(0.031193078324225864, 0.96880692167577409, 0.0, 1.0)

In [199]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0290463179807 0.0311930783242
      Accuracy Rate  0.970953682019  0.968806921676
     True Positives             0.0             0.0
     True Negatives              --             1.0


(0.031193078324225864, 0.96880692167577409, 0.0, 1.0)

In [200]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0290463179807 0.0311930783242
 Accuracy Rate  0.970953682019  0.968806921676
True Positives             0.0             0.0
True Negatives              --             1.0


(0.031193078324225864, 0.96880692167577409, 0.0, 1.0)

In [201]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0333723653396 0.0355950212508
 Accuracy Rate   0.96662763466  0.964404978749
True Positives 0.0279955207167 0.0170316301703
True Negatives              --   0.99490795143


(0.035595021250758953,
 0.96440497874924103,
 0.017031630170316302,
 0.99490795142969057)

In [202]:
# define y
y = totalData.hungry.values
y.shape

(43920,)

In [203]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [204]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Hungry Model Results

In [205]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set  
-------------- -------------- -------------
    Error Rate 0.134270101483 0.13554948391
 Accuracy Rate 0.865729898517 0.86445051609
True Positives            0.0           0.0
True Negatives             --           1.0


(0.13554948391013966, 0.86445051608986034, 0.0, 1.0)

In [206]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.596766848816 0.596387370978
      Accuracy Rate 0.403233151184 0.403612629022
     True Positives 0.858769379845 0.879059350504
     True Negatives             -- 0.329060579456


(0.59638737097753491,
 0.40361262902246509,
 0.87905935050391937,
 0.32906057945566286)

In [207]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate   0.138140775436   0.138357619915
 Accuracy Rate   0.861859224564   0.861642380085
True Positives 0.00654069767442 0.00951847704367
True Negatives               --   0.995258999122


(0.13835761991499695,
 0.86164238008500305,
 0.0095184770436730123,
 0.99525899912203686)

In [208]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate   0.137295081967   0.138281724347
 Accuracy Rate   0.862704918033   0.861718275653
True Positives 0.00290697674419 0.00391937290034
True Negatives               --    0.99622475856


(0.13828172434729813,
 0.86171827565270187,
 0.0039193729003359464,
 0.99622475856014048)

In [209]:
# define y
y = totalData.tired.values
y.shape

(43920,)

In [210]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [211]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Tired Model Results

In [212]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00286234712464 0.00265634486946
 Accuracy Rate   0.997137652875   0.997343655131
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0026563448694596234, 0.99734365513054035, 0.0, 1.0)

In [213]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate 0.00354540723393 0.00318761384335
      Accuracy Rate   0.996454592766   0.996812386157
     True Positives  0.0113636363636  0.0285714285714
     True Negatives               --   0.999391218324


(0.0031876138433515485,
 0.99681238615664847,
 0.028571428571428571,
 0.99939121832432842)

In [214]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00286234712464 0.00265634486946
 Accuracy Rate   0.997137652875   0.997343655131
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0026563448694596234, 0.99734365513054035, 0.0, 1.0)

In [215]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00286234712464 0.00265634486946
 Accuracy Rate   0.997137652875   0.997343655131
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0026563448694596234, 0.99734365513054035, 0.0, 1.0)

In [216]:
# define y
y = totalData.bored.values
y.shape

(43920,)

In [217]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [218]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Bored Model Results

In [219]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0437809003383  0.0395415907711
 Accuracy Rate  0.956219099662   0.960458409229
True Positives             0.0 0.00202429149798
True Negatives              --   0.997792146349


(0.039541590771098971,
 0.96045840922890102,
 0.0020242914979757085,
 0.99779214634915625)

In [220]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate  0.054189435337 0.0506223436551
      Accuracy Rate  0.945810564663  0.949377656345
     True Positives 0.0435126582278 0.0364372469636
     True Negatives              --  0.984939284025


(0.05062234365513054,
 0.94937765634486948,
 0.03643724696356275,
 0.98493928402460185)

In [221]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0411137132449 0.0374924104432
 Accuracy Rate  0.958886286755  0.962507589557
True Positives             0.0             0.0
True Negatives              --             1.0


(0.037492410443230116, 0.96250758955676985, 0.0, 1.0)

In [222]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0420244600572  0.0382513661202
 Accuracy Rate  0.957975539943    0.96174863388
True Positives             0.0 0.00202429149798
True Negatives              --   0.999132628923


(0.03825136612021858,
 0.96174863387978138,
 0.0020242914979757085,
 0.99913262892288279)

# Positive ( Excited, Happy, Calm)


In [223]:
# define y


y = totalData.exhaca.values

# y = totalData.excitement.values
y.shape

(43920,)

In [224]:
# create training and test sets
from sklearn.cross_validation import train_test_split
#
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [225]:
# # feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Positive ( Excited, Happy, Calm) Model Results

In [226]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train, y_train).predict(X_train)
y_pred_test = clf.predict(X_test)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0725995316159 0.0765027322404
 Accuracy Rate  0.927400468384   0.92349726776
True Positives 0.0303676078849 0.0313953488372
True Negatives              --  0.985790841182


(0.07650273224043716,
 0.92349726775956287,
 0.031395348837209305,
 0.98579084118220206)

In [227]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0890905542545 0.0916818457802
      Accuracy Rate  0.910909445746   0.90831815422
     True Positives  0.101225359616  0.109302325581
     True Negatives              --  0.964111724586


(0.091681845780206439,
 0.90831815421979356,
 0.10930232558139535,
 0.96411172458590455)

In [228]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50000, alpha=0.0001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train, y_train).predict(X_train)
y_pred_test = clf.predict(X_test)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0874316939891 0.0915300546448
      Accuracy Rate  0.912568306011  0.908469945355
     True Positives  0.159296750133  0.160465116279
     True Negatives              --   0.96070152647


(0.091530054644808748,
 0.90846994535519121,
 0.16046511627906976,
 0.96070152646963303)

In [229]:
# print(clf.coef_)
# print(clf.intercept_)
# print(X_train_std[0:5])
# print(X_train[0:5])
# heartrate, steps, calories, gsr, skintemp, airtemp, weekday, hour, curmin


In [230]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0622235232891  0.0664086217365
 Accuracy Rate  0.937776476711   0.933591378264
True Positives 0.0042621204049 0.00348837209302
True Negatives              --   0.998538486522


(0.066408621736490592,
 0.93359137826350946,
 0.0034883720930232558,
 0.99853848652159793)

In [231]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0923106947697 0.0961596842744
 Accuracy Rate   0.90768930523  0.903840315726
True Positives  0.151305274374  0.145348837209
True Negatives              --  0.956804157194


(0.09615968427443837,
 0.90384031572556167,
 0.14534883720930233,
 0.95680415719389411)

# Negative ( Anxious, Sad, Angry)

In [232]:
# define y
y = totalData.ansaan.values
y.shape

(43920,)

In [233]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [234]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Negative ( Anxious, Sad, Angry) Model Results

In [235]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00162633359355 0.00129022465088
 Accuracy Rate   0.998373666406   0.998709775349
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0012902246508803886, 0.99870977534911964, 0.0, 1.0)

In [236]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set    
------------------- --------------- ----------------
         Error Rate 0.0017889669529 0.00144201578628
      Accuracy Rate  0.998211033047   0.998557984214
     True Positives             0.0              0.0
     True Negatives              --   0.999848012767


(0.0014420157862780813, 0.99855798421372188, 0.0, 0.99984801276692759)

In [237]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00162633359355 0.00129022465088
 Accuracy Rate   0.998373666406   0.998709775349
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0012902246508803886, 0.99870977534911964, 0.0, 1.0)

In [238]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00162633359355 0.00129022465088
 Accuracy Rate   0.998373666406   0.998709775349
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0012902246508803886, 0.99870977534911964, 0.0, 1.0)

# Happy/Calm 

In [239]:
# define y
y = totalData.haca.values
y.shape

(43920,)

In [240]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [241]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Happy/Calm  Model Results

In [242]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate  0.20706479313 0.209699453552
 Accuracy Rate  0.79293520687 0.790300546448
True Positives 0.164728682171 0.159574468085
True Negatives             -- 0.889201053556


(0.20969945355191258,
 0.79030054644808745,
 0.15957446808510639,
 0.88920105355575063)

In [243]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set  
------------------- -------------- -------------
         Error Rate 0.134270101483 0.13554948391
      Accuracy Rate 0.865729898517 0.86445051609
     True Positives            0.0           0.0
     True Negatives             --           1.0


(0.13554948391013966, 0.86445051608986034, 0.0, 1.0)

In [244]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate   0.138140775436   0.138357619915
 Accuracy Rate   0.861859224564   0.861642380085
True Positives 0.00654069767442 0.00951847704367
True Negatives               --   0.995258999122


(0.13835761991499695,
 0.86164238008500305,
 0.0095184770436730123,
 0.99525899912203686)

In [245]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set      Test Set   
-------------- -------------- ---------------
    Error Rate 0.806075982305  0.801760777171
 Accuracy Rate 0.193924017695  0.198239222829
True Positives 0.989825581395  0.991041433371
True Negatives             -- 0.0739244951712


(0.80176077717061323,
 0.19823922282938677,
 0.99104143337066064,
 0.073924495171202809)

# Anxious/Angry

In [246]:
# define y
y = totalData.anan.values
y.shape

(43920,)

In [247]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [248]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [249]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00289487379651 0.00258044930176
 Accuracy Rate   0.997105126203   0.997419550698
True Positives              0.0  0.0285714285714
True Negatives               --              1.0


(0.0025804493017607772, 0.99741955069823918, 0.028571428571428571, 1.0)

In [250]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate 0.00468384074941 0.00447783849423
      Accuracy Rate   0.995316159251   0.995522161506
     True Positives  0.0113636363636  0.0857142857143
     True Negatives               --   0.997945361845


(0.0044778384942319371,
 0.99552216150576811,
 0.085714285714285715,
 0.99794536184460847)

In [251]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00286234712464 0.00265634486946
 Accuracy Rate   0.997137652875   0.997343655131
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0026563448694596234, 0.99734365513054035, 0.0, 1.0)

In [252]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00286234712464 0.00265634486946
 Accuracy Rate   0.997137652875   0.997343655131
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0026563448694596234, 0.99734365513054035, 0.0, 1.0)

# Excited/Happy

In [253]:
# define y
y = totalData.exha.values
y.shape

(43920,)

In [254]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [255]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Excited/Happy Model Values

In [256]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0290463179807 0.0311930783242
 Accuracy Rate  0.970953682019  0.968806921676
True Positives             0.0             0.0
True Negatives              --             1.0


(0.031193078324225864, 0.96880692167577409, 0.0, 1.0)

In [257]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate  0.0296968514182  0.0318002428658
      Accuracy Rate   0.970303148582   0.968199757134
     True Positives 0.00111982082867 0.00243309002433
     True Negatives               --   0.999294947121


(0.031800242865816633,
 0.96819975713418338,
 0.0024330900243309003,
 0.99929494712103406)

In [258]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0290463179807 0.0311930783242
 Accuracy Rate  0.970953682019  0.968806921676
True Positives             0.0             0.0
True Negatives              --             1.0


(0.031193078324225864, 0.96880692167577409, 0.0, 1.0)

In [259]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0294366380432 0.0313448694596
 Accuracy Rate  0.970563361957   0.96865513054
True Positives             0.0             0.0
True Negatives              --  0.999843321582


(0.031344869459623559, 0.96865513054037644, 0.0, 0.99984332158245204)

# Anxious/Sad

In [260]:
# define y
y = totalData.ansa.values
y.shape

(43920,)

In [261]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [262]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Anxious/Sad Model Results

In [263]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0411137132449 0.0374924104432
 Accuracy Rate  0.958886286755  0.962507589557
True Positives             0.0             0.0
True Negatives              --             1.0


(0.037492410443230116, 0.96250758955676985, 0.0, 1.0)

In [264]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0412438199323 0.0374924104432
      Accuracy Rate  0.958756180068  0.962507589557
     True Positives             0.0             0.0
     True Negatives              --             1.0


(0.037492410443230116, 0.96250758955676985, 0.0, 1.0)

In [265]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0411137132449 0.0374924104432
 Accuracy Rate  0.958886286755  0.962507589557
True Positives             0.0             0.0
True Negatives              --             1.0


(0.037492410443230116, 0.96250758955676985, 0.0, 1.0)

In [266]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0411137132449 0.0374924104432
 Accuracy Rate  0.958886286755  0.962507589557
True Positives             0.0             0.0
True Negatives              --             1.0


(0.037492410443230116, 0.96250758955676985, 0.0, 1.0)